In [1]:
# -*- coding: utf-8 -*-
"""
Created on Fri May 20 16:36:06 2016

@author: Aditya
"""

#Rang Competition
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.cross_validation import train_test_split

#Loading the data
train = pd.read_csv("Train.csv")
test = pd.read_csv("Test.csv")

#training
train.info()
#Missing values in any cell
train.isnull().values.ravel().sum()
#rows with missing values
train.shape[0] - train.dropna().shape[0]
cols = train.columns
#numeric columns
num_cols = train._get_numeric_data().columns
#non-numerical columns
list(set(cols)-set(num_cols))
#encoding the categorical variable
cat_encoder = LabelEncoder()
cat1_encoder = LabelEncoder()
cat2_encoder = LabelEncoder()
cat3_encoder = LabelEncoder()
cat4_encoder = LabelEncoder()
cat_encoder.fit(train["Trans24"])
cat1_encoder.fit(train["Trans25"])
cat2_encoder.fit(train["Trans26"])
cat3_encoder.fit(train["Trans27"])
cat4_encoder.fit(train["Cust_status"])
train["Trans24"] = cat_encoder.transform(train["Trans24"])
train["Trans25"] = cat1_encoder.transform(train["Trans25"])
train["Trans26"] = cat2_encoder.transform(train["Trans26"])
train["Trans27"] = cat3_encoder.transform(train["Trans27"])
train["Cust_status"] = cat4_encoder.transform(train["Cust_status"])
cat_encoder = LabelEncoder()
cat1_encoder = LabelEncoder()
cat2_encoder = LabelEncoder()
cat3_encoder = LabelEncoder()
cat4_encoder = LabelEncoder()
cat_encoder.fit(test["Trans24"])
cat1_encoder.fit(test["Trans25"])
cat2_encoder.fit(test["Trans26"])
cat3_encoder.fit(test["Trans27"])
cat4_encoder.fit(test["Cust_status"])
test["Trans24"] = cat_encoder.transform(test["Trans24"])
test["Trans25"] = cat1_encoder.transform(test["Trans25"])
test["Trans26"] = cat2_encoder.transform(test["Trans26"])
test["Trans27"] = cat3_encoder.transform(test["Trans27"])
test["Cust_status"] = cat4_encoder.transform(test["Cust_status"])
#Fill missing values with mean in all columns 
train = train.where(pd.notnull(train), train.mean(), axis = "columns")
test = test.where(pd.notnull(test), test.mean(), axis = "columns")
#Splitting the training set
train_train, validation = train_test_split(train, random_state = 30)
#selecting required columns
train_columns = list(train.columns[2:256].values)
test_columns = list(test.columns[2:].values)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25766 entries, 0 to 25765
Columns: 257 entries, Cust_id to Active_Customer
dtypes: float64(234), int64(17), object(6)
memory usage: 50.5+ MB


In [2]:
#Basic logistic model
Model = GradientBoostingClassifier(loss='deviance', learning_rate=0.135, n_estimators=210, 
                                   subsample=0.80, min_samples_split=2, min_samples_leaf=1, 
                                   min_weight_fraction_leaf=0.0, max_depth=3, init=None, 
                                   random_state=50, max_features=None, verbose=1, 
                                   max_leaf_nodes=None, warm_start=False, presort='auto')
Model.fit(train_train[train_columns],train_train["Active_Customer"])
Model.predict(validation[train_columns])
Model.score(validation[train_columns],validation["Active_Customer"])
Model.feature_importances_
predict = Model.predict(test[test_columns])

#writing to csv
result = pd.DataFrame(predict, columns=(["Active_Customer"]), index = test["Cust_id"])
result.to_csv("submission_110.csv")

#Model = GradientBoostingClassifier(loss='deviance', learning_rate=0.05, n_estimators=800, 
#                                   subsample=1.0, min_samples_split=10, min_samples_leaf=10, 
#                                   min_weight_fraction_leaf=0.0, max_depth=5, init=None, 
#                                   random_state=None, max_features="auto", verbose=0, 
#                                   max_leaf_nodes=None, warm_start=False, presort='auto')
#Model.fit(train_train[train_columns],train_train["Active_Customer"])
#Model.predict(validation[train_columns])
#Model.score(validation[train_columns],validation["Active_Customer"])
#Model.feature_importances_
#predict = Model.predict(test[test_columns])
#Model = GradientBoostingClassifier(loss='deviance', learning_rate=0.125, n_estimators=250, 
#                                  subsample=0.85, min_samples_split=2, min_samples_leaf=1, 
#                                  min_weight_fraction_leaf=0.0, max_depth=3, init=None, 
#                                  random_state=50, max_features="auto", verbose=1, 
#                                  max_leaf_nodes=None, warm_start=True, presort="auto")
#Model.fit(train_train[train_columns],train_train["Active_Customer"])
#Model.predict(validation[train_columns])
#Model.score(validation[train_columns],validation["Active_Customer"])
#Model.feature_importances_
#predict = Model.predict(test[test_columns])

      Iter       Train Loss      OOB Improve   Remaining Time 
         1           1.3477           0.0368            5.34m
         2           1.3186           0.0281            3.87m
         3           1.2933           0.0221            3.17m
         4           1.2760           0.0181            2.89m
         5           1.2615           0.0137            2.64m
         6           1.2487           0.0119            2.41m
         7           1.2372           0.0086            2.24m
         8           1.2301           0.0081            2.10m
         9           1.2209           0.0052            2.01m
        10           1.2171           0.0059            1.93m
        20           1.1857           0.0004            1.53m
        30           1.1703           0.0003            1.40m
        40           1.1597          -0.0001            1.22m
        50           1.1504           0.0000            1.08m
        60           1.1385          -0.0010           58.24s
       

In [4]:
import xgboost as xgb

In [10]:
#XGBoost model

params = {
    'learning_rate': 0.135,
    'max_depth': 3,
    'min_child_weight': 1,
    'n_estimators': 20,
    'nthread': -1,
    'objective': 'binary:logistic',
    'seed': 50,
    'subsample': 0.8
}

Model = xgb.XGBClassifier()
Model.set_params(**params)
Model.fit(train_train[train_columns],train_train["Active_Customer"])
Model.predict(validation[train_columns])
Model.score(validation[train_columns],validation["Active_Customer"])
#Model.feature_importances_
predict = Model.predict(test[test_columns])

#writing to csv
result = pd.DataFrame(predict, columns=(["Active_Customer"]), index = test["Cust_id"])
result.to_csv("submission_113.csv")